In [1]:
import math

In [2]:
import numpy as np
import pandas as pd

class Node:

  def __init__(self, value, children:list=None, parent_choice=None):
    self.value = value
    if children and type(children) != list:
      raise TypeError("children must be of type list")

    self.children = children or []
    self.parent_choice = parent_choice

  def is_leaf(self):
    return not(len(self.children))

  def add_child(self, subtree, parent_choice):
    if type(subtree) == Tree:
      subtree.root.parent_choice = parent_choice
      self.children.append(subtree.root)
    else:
      self.children.append(Node(subtree, parent_choice=parent_choice))

  def __str__(self):
    return f"{self.value}"

  def __repr__(self):
    return f"{self.value}"


class Tree:

  def __init__(self, root:Node):
    if type(root) != Node:
      raise TypeError(f"root must be of type {Node.__name__}")

    self.root = root

  def __str__(self):
    Tree.__print_node(self.root)
    return ""

  @staticmethod
  def __print_node(node):
        if not node.is_leaf():
            print("node:", node.value)
            print("children:")
            for i, child in enumerate(node.children):
                print("   ", "parent_choice:", child.parent_choice, "|", f"ch{i}:", child.value)

            print("----------------------")

            for child in node.children:
                Tree.__print_node(child)

In [3]:
rest_df = pd.DataFrame({
    'alt': [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
    'bar': [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1],
    'fri': [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
    'hun': [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'pat': [
        'some', 'full', 'some', 'full', 'full',
        'some', 'none', 'some', 'full', 'full',
        'none', 'full'
    ],
    'price': [3, 1, 1, 1, 3, 2, 1, 2, 1, 3, 1, 1],
    'rain': [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0],
    'res': [1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
    'type': ['0', '1', '2', '1',
             '0', '3', '2',
             '1', '2', '3',
             '1', '2'
             ],
    'est': [1, 2, 1, 3, 4, 1, 1, 1, 4, 3, 1, 2],
    'result': [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
})

In [4]:
est_map = {
    1: '0-10',
    2: '30-60',
    3: '10-30',
    4: '>60'
}

In [5]:
rest_df.columns

Index(['alt', 'bar', 'fri', 'hun', 'pat', 'price', 'rain', 'res', 'type',
       'est', 'result'],
      dtype='object')

In [92]:
def prune(attr:str, examples:pd.DataFrame):
  # check for plurality of more than 95%, if so, make it leaf. and return its value.

  ones = len(examples[(examples.iloc[:,-1] == 1)])
  zeros = len(examples[(examples.iloc[:,-1] == 0)])

  if (zeros / len(examples)) > 0.8:
    return 0
  elif (ones / len(examples)) > 0.8:
    return 1
  else:
    return

In [7]:
data = {'A': [1, 2, 3], 'B': ['a', 'b', 'c']}
df = pd.DataFrame(data)

value = df.loc[:, 'B']
print(value)

0    a
1    b
2    c
Name: B, dtype: object


In [9]:
from math import log2

def calc_B(val):
  if val == 1 or val == 0:
    return 0

  return -(val * log2(val) + (1 - val) * log2(1 - val))


def importance(attr:str, examples:pd.DataFrame):
  # based on entropy

  # gain(A) = B(p/(p+n)) - Remainder(A)
  # Remainder(A) = SIGMA(k=1, k=d)((pk + nk)/(p + n) * B(pk/(pk + nk)))
  # B(q) = -(q log2(q) + (1 - q) log2(1-q))

  branches = examples[attr].unique()
  ReA = 0
  p = len(examples[(examples.iloc[:,-1] == 1)])
  n = len(examples[(examples.iloc[:,-1] == 0)])
  val = p / (p+n)
  B = calc_B(val)

  for k in range(len(branches)):
    temp = examples[(examples[attr] == branches[k])]
    pk = len(temp[(temp.iloc[:,-1] == 1)])
    nk = len(temp[(temp.iloc[:,-1] == 0)])
    sigk = ((pk + nk) / (p + n)) * calc_B(pk / (pk + nk))
    ReA += sigk

  return B - ReA

def calc_gini(zeros, ones) -> float:
  all_ = zeros + ones
  sigma = (zeros / all_) ** 2 + (ones / all_) ** 2
  return 1 - sigma

def importance_gini_index(attr:str, examples:pd.DataFrame):
  # based on Gini index

  # gini = 1 - SIGMA(i)(Pi ** 2)
  zeros = len(examples[(examples.iloc[:,-1] == 0)])
  ones = len(examples[(examples.iloc[:,-1] == 1)])
  parent_gini = calc_gini(zeros, ones)

  gini_sum = 0

  branches = examples[attr].unique()

  for choice in branches:
    branch_exs = examples[(examples[attr] == choice)]
    zeros = len(branch_exs[(branch_exs.iloc[:,-1] == 0)])
    ones = len(branch_exs[(branch_exs.iloc[:,-1] == 1)])
    gini_sum += calc_gini(zeros, ones)

  gini_avg = gini_sum / len(branches)

  return parent_gini - gini_avg


In [10]:
def plurality_value(values:pd.DataFrame):
  length = len(values)
  return int(len(values[(values.iloc[:,-1] == 1)]) > (length / 2))

In [11]:
def decision_tree_learn(examples:pd.DataFrame, attributes:pd.Series, parent_examples:pd.DataFrame, attribute_values:dict[str, list], use_gini=False):

  if len(examples) == 0:
    if len(parent_examples):
      return plurality_value(parent_examples)
    else:
      return None

  elif len(examples.iloc[:,-1].unique()) == 1:
    return int(examples.iloc[0, -1])

  elif len(attributes) == 0:
    return plurality_value(examples)

  else:

    selected_attr = max(range(len(attributes)), key=
                        lambda index:
                          importance_gini_index(attributes[index], examples)
                          if use_gini else
                          importance(attributes[index], examples)
                        )
    selected_attr = attributes.pop(selected_attr)

    if (val := prune(selected_attr, examples)) is not None:
      return val

    A = Node(selected_attr)
    tree = Tree(A)

    choices = attribute_values[selected_attr]

    for choice in choices:
      child_examples = examples[(examples[selected_attr] == choice)]
      subtree = decision_tree_learn(child_examples, attributes, examples, attribute_values, use_gini)
      A.add_child(subtree, choice)

    return tree


In [12]:
importance_gini_index('type', rest_df)

0.0

In [13]:
importance('type', rest_df)

1.1102230246251565e-16

In [14]:
def predict(tree:Tree, example:pd.Series):
  node = tree.root

  while not node.is_leaf():
    attr_value = example[node.value]

    for child in node.children:
      if child.parent_choice == attr_value:
        node = child
        break

  return node.value

In [50]:
def accuracy(tree: Tree, test_df: pd.DataFrame):
  predicted_res = []
  for index, row in test_df.iterrows():
      predicted_res.append(predict(tree, row))

  misses = 0
  for i in range(len(predicted_res)):
    if predicted_res[i] != test_df.iloc[i,-1]:
      misses += 1

  return 100 * (1 - misses / len(test_df))

In [51]:
TRAIN_PORTION = 0.8

rest_df_ones = rest_df[rest_df["result"] == 1]
rest_df_zeros = rest_df[rest_df["result"] == 0]

rest_rows_to_select_ones = int(len(rest_df_ones) * TRAIN_PORTION)
rest_rows_to_select_zeros = int(len(rest_df_zeros) * TRAIN_PORTION)

rest_train_data_ones = rest_df_ones.sample(n=rest_rows_to_select_ones, random_state=42)
rest_train_data_zeros = rest_df_zeros.sample(n=rest_rows_to_select_zeros, random_state=42)

In [52]:
rest_train_df = pd.concat([rest_train_data_ones, rest_train_data_zeros])
rest_test_df = rest_df.drop(rest_train_df.index)

In [53]:
attr_values = {}
for col in rest_df.columns:
  attr_values[col] = rest_df[col].unique()

tree = decision_tree_learn(rest_train_df, list(rest_train_df.columns)[:-1], pd.DataFrame([]), attr_values)

print("Acc(entropy):", accuracy(tree, rest_test_df))

Acc(entropy): 75.0


In [54]:
print(tree)

node: pat
children:
    parent_choice: some | ch0: 1
    parent_choice: full | ch1: type
    parent_choice: none | ch2: 0
----------------------
node: type
children:
    parent_choice: 0 | ch0: 0
    parent_choice: 1 | ch1: fri
    parent_choice: 2 | ch2: 1
    parent_choice: 3 | ch3: 0
----------------------
node: fri
children:
    parent_choice: 0 | ch0: 0
    parent_choice: 1 | ch1: 1
----------------------



In [55]:
tree = decision_tree_learn(rest_train_df, list(rest_train_df.columns)[:-1], pd.DataFrame([]), attr_values, True)

print("Acc(gini):", accuracy(tree, rest_test_df))

Acc(gini): 75.0


In [56]:
print(tree)

node: pat
children:
    parent_choice: some | ch0: 1
    parent_choice: full | ch1: type
    parent_choice: none | ch2: 0
----------------------
node: type
children:
    parent_choice: 0 | ch0: 0
    parent_choice: 1 | ch1: fri
    parent_choice: 2 | ch2: 1
    parent_choice: 3 | ch3: 0
----------------------
node: fri
children:
    parent_choice: 0 | ch0: 0
    parent_choice: 1 | ch1: 1
----------------------



In [57]:
air_df = pd.read_csv("Airplane.csv")
air_df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [58]:
air_df['satisfaction'].unique()

array(['neutral or dissatisfied', 'satisfied'], dtype=object)

In [59]:
air_df['satisfaction'] = air_df['satisfaction'].map({"neutral or dissatisfied": 0, "satisfied": 1})

In [60]:
air_df['satisfaction'].unique()

array([0, 1])

In [61]:
air_df.shape[0]

103904

In [62]:
max(air_df['Flight Distance'])

4983

In [63]:
min(air_df['Flight Distance'])

31

In [101]:
def spectrum_to_descrete_mapper(value: int, _range, chunk_number: int = 10):
  """
  calculates the location of the <value> in the range <_range>
  and returns a number between 0 and chunk_number -1
  """
  percent = value / _range * chunk_number
  return math.floor(percent)

In [102]:
max_flight_distance = max(air_df["Flight Distance"])
min_flight_distance = min(air_df["Flight Distance"])
flight_distance_range = max_flight_distance - min_flight_distance
air_df["Flight Distance"] = air_df['Flight Distance'].map(lambda value: spectrum_to_descrete_mapper(value, flight_distance_range, 10))

In [103]:
air_df["Customer Type"] = air_df["Customer Type"].map({"Loyal Customer": 1, "disloyal Customer": 0})

In [67]:
air_df["Gender"] = air_df["Gender"].map({"Male": 1, "Female": 0})

In [68]:
max_age = max(air_df['Age'])
min_age = min(air_df['Age'])
age_range = max_age - min_age
air_df["Age"] = air_df["Age"].map(lambda value: spectrum_to_descrete_mapper(value, age_range, 10))

In [69]:
air_df["Type of Travel"] = air_df["Type of Travel"].map({"Personal Travel": 1, "Business travel": 0})

In [70]:
air_df["Class"] = air_df["Class"].map({"Eco Plus": 0, "Business": 1, "Eco": 2})

In [71]:
min(air_df["Departure Delay in Minutes"])

0

In [72]:
max(air_df["Departure Delay in Minutes"])

1592

In [73]:
air_df["Departure Delay in Minutes"].mean()

14.815618263012011

In [74]:
max_delay = max(air_df['Departure Delay in Minutes'])
min_delay = min(air_df['Departure Delay in Minutes'])
delay_range = max_delay - min_delay
air_df["Departure Delay in Minutes"] = air_df["Departure Delay in Minutes"]\
.map(lambda value: spectrum_to_descrete_mapper(value, delay_range, 5))

In [75]:
min(air_df["Arrival Delay in Minutes"])

0.0

In [76]:
max(air_df["Arrival Delay in Minutes"])

1584.0

In [77]:
air_df["Arrival Delay in Minutes"].mean()

15.178678301832152

In [78]:
arrival_delay_in_minutes_mean = air_df["Arrival Delay in Minutes"].mean()
air_df["Arrival Delay in Minutes"] = air_df["Arrival Delay in Minutes"]\
.map(lambda value: value if value > 0 else arrival_delay_in_minutes_mean)

In [79]:
max_delay = max(air_df['Arrival Delay in Minutes'])
min_delay = min(air_df['Arrival Delay in Minutes'])
delay_range = max_delay - min_delay
air_df["Arrival Delay in Minutes"] = air_df["Arrival Delay in Minutes"]\
.map(lambda value: spectrum_to_descrete_mapper(value, delay_range, 5))

In [80]:
air_df.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [81]:
list_of_required_cols = list(air_df.columns)[2:]
air_df = air_df[list_of_required_cols]

In [94]:
TRAIN_PORTION = 0.8

air_df_ones = air_df[air_df["satisfaction"] == 1]
air_df_zeros = air_df[air_df["satisfaction"] == 0]

rows_to_select_ones = int(len(air_df_ones) * TRAIN_PORTION)
rows_to_select_zeros = int(len(air_df_zeros) * TRAIN_PORTION)

train_data_ones = air_df_ones.sample(n=rows_to_select_ones, random_state=10)
train_data_zeros = air_df_zeros.sample(n=rows_to_select_zeros, random_state=10)

In [95]:
train_df = pd.concat([train_data_ones, train_data_zeros])
test_df = air_df.drop(train_df.index)

In [96]:
attr_values = {}
for col in air_df.columns:
  attr_values[col] = air_df[col].unique()

tree = decision_tree_learn(train_df, list(train_df.columns)[:-1], pd.DataFrame([]), attr_values, False)
print("Acc(entropy):", accuracy(tree, test_df))

Acc(entropy): 82.75828882151966


In [97]:
print(tree)

node: Online boarding
children:
    parent_choice: 3 | ch0: 0
    parent_choice: 5 | ch1: 1
    parent_choice: 2 | ch2: 0
    parent_choice: 1 | ch3: 0
    parent_choice: 4 | ch4: Inflight entertainment
    parent_choice: 0 | ch5: 1
----------------------
node: Inflight entertainment
children:
    parent_choice: 5 | ch0: Leg room service
    parent_choice: 1 | ch1: 0
    parent_choice: 2 | ch2: 0
    parent_choice: 3 | ch3: 0
    parent_choice: 4 | ch4: 1
    parent_choice: 0 | ch5: 0
----------------------
node: Leg room service
children:
    parent_choice: 3 | ch0: Food and drink
    parent_choice: 5 | ch1: 1
    parent_choice: 4 | ch2: 1
    parent_choice: 2 | ch3: 0
    parent_choice: 1 | ch4: 0
    parent_choice: 0 | ch5: 0
----------------------
node: Food and drink
children:
    parent_choice: 5 | ch0: Age
    parent_choice: 1 | ch1: 0
    parent_choice: 2 | ch2: 0
    parent_choice: 4 | ch3: 0
    parent_choice: 3 | ch4: 0
    parent_choice: 0 | ch5: 1
----------------------
no

In [98]:
tree2 = decision_tree_learn(train_df, list(train_df.columns)[:-1], pd.DataFrame([]), attr_values, True)
print("Acc(gini-index):", accuracy(tree2, test_df))

Acc(gini-index): 81.40609210336365


In [89]:
print(tree2)

node: Departure Delay in Minutes
children:
    parent_choice: 0 | ch0: Inflight wifi service
    parent_choice: 1 | ch1: 0
    parent_choice: 4 | ch2: 1
    parent_choice: 2 | ch3: 0
    parent_choice: 3 | ch4: 0
    parent_choice: 5 | ch5: 0
----------------------
node: Inflight wifi service
children:
    parent_choice: 3 | ch0: Gate location
    parent_choice: 2 | ch1: 0
    parent_choice: 4 | ch2: 1
    parent_choice: 1 | ch3: 0
    parent_choice: 5 | ch4: 1
    parent_choice: 0 | ch5: 1
----------------------
node: Gate location
children:
    parent_choice: 1 | ch0: Flight Distance
    parent_choice: 3 | ch1: 0
    parent_choice: 2 | ch2: 0
    parent_choice: 5 | ch3: 0
    parent_choice: 4 | ch4: 0
    parent_choice: 0 | ch5: 0
----------------------
node: Flight Distance
children:
    parent_choice: 0 | ch0: Age
    parent_choice: 2 | ch1: 0
    parent_choice: 1 | ch2: 0
    parent_choice: 4 | ch3: 0
    parent_choice: 5 | ch4: 0
    parent_choice: 3 | ch5: 0
    parent_choice: 6

In [99]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X = air_df.iloc[:,:-1]
y = air_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
tree = DecisionTreeClassifier(random_state=10)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

accuracy_value = tree.score(X_test, y_test)

print(f"Accuracy: {accuracy_value * 100}")

Accuracy: 94.4516625763919
